In [164]:
import requests
import pandas as pd

In [34]:
URL_SERVER = "http://localhost:3030"
URL_DATASET = URL_SERVER+"/final_project"
URL_GRAPHSTORE = URL_DATASET+"/data"
URL_QUERY = URL_DATASET + "/sparql"

In [71]:
def ontology_uploader(file):
    data_ttl = open(file).read()
    res = requests.post(URL_GRAPHSTORE, data=data_ttl, headers={'Content-Type': 'text/turtle'})
    print("req", res)
    if res.status_code != 204:
        res.raise_for_status()
    else:
        print("Status response",res)

In [68]:
def json_uploader(file):
    data_json = open(file).read()
    res = requests.post(URL_GRAPHSTORE, data=data_json, headers={'Content-Type': 'application/ld+json'})
    #checking if the file was uploaded correctly:
    if res.status_code != 204 and res.status_code != 200:
        res.raise_for_status()
    else:
        print("Status response",res,file,"correctly uploaded !")
        
    return res

In [54]:
def CleanDB():
    query = """
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX ns: <http://www.owl-ontologies.com/unnamed.owl#> 
        DELETE
        {
            ?s ?p ?o
        }
        WHERE
        {
            ?s ?p ?o
        }
    """
    res = requests.post(URL_DATASET, data=query, headers={'Content-type': 'application/sparql-update'})
    
    #checking if the file was uploaded correctly:
    if res.status_code != 204 :
        res.raise_for_status()
    else:
        print("Status response",res)
        print("Dataset correctly cleared !")
    
    return res
    


In [122]:
def find(query):
    result = requests.post(URL_QUERY, data=query, headers={'Content-type': 'application/sparql-query'})
    
    if result.status_code != 204 :
        result.raise_for_status()
    else:
        print("Status response",result)
        print("Dataset correctly cleared !")

    return result

In [86]:
def main():
    
    CleanDB()

    ontology_uploader("D:/ESILV_Cours/S8/Datamining_Semantics/ProjetFinal/ontology/ontology_projet_final.ttl")

    json_uploader("monuments.jsonld")
    json_uploader("velo.jsonld") #dynamic values
    json_uploader("musees.jsonld")

In [87]:
main()
#CleanDB()

Status response <Response [204]>
Dataset correctly cleared !
req <Response [200]>
Status response <Response [200]> monuments.jsonld correctly uploaded !
Status response <Response [200]> velo.jsonld correctly uploaded !
Status response <Response [200]> musees.jsonld correctly uploaded !


In [123]:
def query_to_df(query):
    data = find(query).json()
    l = []
    for elem in data["results"]["bindings"]:
        d = {}
        for k,v in elem.items():
            d[k] = v["value"]
        l.append(d)
    return pd.DataFrame(l)
    

In [161]:
def query_bool(query):
    data = find(query)
    return data.json()["boolean"]

In [155]:
def query_string(query):
    data = find(query)
    return str(data.content).split('\\n')

In [165]:
query = """PREFIX j.0: <http://www.owl-ontologies.com/unnamed.owl#>
SELECT ?nom_musee ?adresse_musee WHERE { ?subject j.0:nom_musee ?nom_musee;
j.0:adresse_musee ?adresse_musee }
"""
query2 = """PREFIX p: <http://www.owl-ontologies.com/unnamed.owl#>
DESCRIBE ?subject WHERE {?subject p:id_velo "10111"}
"""
query3 = """PREFIX j.0: <http://www.owl-ontologies.com/unnamed.owl#>
ASK {?subject j.0:nom_mon "Maison Renaissance" ; j.0:historique ?historique}"""
a = query_to_df(query)

In [168]:
list(a.iloc[:,0])

['musée des beaux-arts',
 'musée Gadagne',
 'musée Guimet',
 'musée des arts décoratifs',
 "muséum d'histoire naturelle",
 "centre d'histoire de la résistance",
 'Lugdunum, musée et théâtres romains',
 'musée historique des tissus',
 'musée des hospices civils de Lyon',
 'musée des Confluences',
 "musée d'art contemporain",
 'musée des sapeurs-pompiers de Lyon',
 "musée de l'imprimerie et de la communication graphique"]